This notebook trains a surrogate model using TensorFlow.  The resulting model is then saved and converted to a TensorFlowJS format.  The model is then loaded and used to make predictions on new data on a webpage.

To run this notebooks, use the `ws-env` virtual environment, which can be built using the `environment.yml` file located in the same directory as this notebook.

In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [ ]:
# Read in training data
df = pd.read_csv('ExampleData.csv')
# Split the X and y variables
X = df[["age_effect", "initial_effect", "final_effect", "mort_effect", "prod_effect", "fert_effect", "discount_rate"]].values
y = df[["NPV"]].values

In [ ]:
# train-test split for model evaluation
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, train_size=0.7, shuffle=True
# )
# In this case, don't split since it's important
# to use the whole sample to ensure the outer edges
# of the parameter space are covered
X_train = X
y_train = y

In [4]:
# Layer setting
num_input = X.shape[1]
num_hidden1 = 10 * num_input
num_hidden2 = 10 * num_input
num_output = y.shape[1]
layers_dim = [num_input, num_hidden1, num_hidden2, num_output]
print("Dimensions of each layer are {}".format(layers_dim))

Dimensions of each layer are [7, 70, 70, 2]


In [ ]:
# Define the neural network
# We use [Keras](https://www.tensorflow.org/guide/keras) to define the
# neural network
# Create a normalization layer
norm_layer = tf.keras.layers.Normalization(input_shape=[num_input,], axis=None)
# Initialize the weights
initializer = tf.keras.initializers.HeUniform()

# Adapt the layer to your training data
norm_layer.adapt(X_train)
nn = tf.keras.Sequential(
    [
        keras.layers.Input(shape=(num_input,)),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_hidden2, activation="gelu", kernel_initializer=initializer),
        keras.layers.Dense(num_output, kernel_initializer=initializer),
    ]
)
print(nn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                560       
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 2)                 142       
                                                                 
Total params: 5672 (22.16 KB)
Trainable params: 5672 (22.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
nn.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['mean_squared_error'])

In [ ]:
# It's important to use at least 10_000 epochs to ensure the model
# fit is good
nn.fit(X_train, y_train, epochs=20_000)

Epoch 1/200
354/354 [==============================] - 0s 600us/step - loss: 3.3362 - accuracy: 0.6287
Epoch 2/200
354/354 [==============================] - 0s 668us/step - loss: 0.3321 - accuracy: 0.7201
Epoch 3/200
354/354 [==============================] - 0s 617us/step - loss: 0.2244 - accuracy: 0.7265
Epoch 4/200
354/354 [==============================] - 0s 661us/step - loss: 0.1523 - accuracy: 0.7285
Epoch 5/200
354/354 [==============================] - 0s 606us/step - loss: 0.1036 - accuracy: 0.7429
Epoch 6/200
354/354 [==============================] - 0s 583us/step - loss: 0.0779 - accuracy: 0.7479
Epoch 7/200
354/354 [==============================] - 0s 576us/step - loss: 0.0550 - accuracy: 0.7502
Epoch 8/200
354/354 [==============================] - 0s 585us/step - loss: 0.0444 - accuracy: 0.7545
Epoch 9/200
354/354 [==============================] - 0s 586us/step - loss: 0.0391 - accuracy: 0.7516
Epoch 10/200
354/354 [==============================] - 0s 646us/step - l

In [8]:
nn.evaluate(X_test,  y_test, verbose=2)

152/152 - 0s - loss: 2.9100e-04 - accuracy: 0.9990 - 123ms/epoch - 807us/step


[0.0002909964823629707, 0.9989688396453857]

In [ ]:
# See how well the model is doing in terms of predictions
# on our dataset
predictions = nn.predict(X)
# add predictions to original df
df["NPV_pred"] = predictions[:, 0]
df["NPV_diff"] = df["NPV"] - df["NPV_pred"]
print('The maximum difference between the actual NPV and the predicted NPV is: {}'.format(df["NPV_diff"].max()))
print('The minimum difference between the actual NPV and the predicted NPV is: {}'.format(df["NPV_diff"].min()))
print('The mean absolute difference between the actual NPV and the predicted NPV is: {}'.format(np.absolute(df["NPV_diff"]).mean()))
print('The S.D. in the predicted value is: {}'.format(df["NPV_pred"].std()))

In [9]:
# save full model - not just weights
tf.keras.Model.save(nn, "SL_model_full.h5", save_format="h5")

/Users/jason.debacker/anaconda3/envs/dynviz-dev/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
!tensorflowjs_converter --input_format=keras SL_model_full.h5 ./tf_model/